In [92]:
import numpy as np
import pandas as pd
import geopandas as gpd
import folium
from shapely.geometry import Point, LineString, Polygon, box

In [93]:
bpd = pd.read_csv('berkeleypd.csv')

In [94]:
# get data set with selected crimes
combined = bpd[(bpd['CVLEGEND'] == 'ASSAULT') | (bpd['CVLEGEND'] == 'ROBBERY')]
print(combined.shape)

# drop any entries without a location
count = 0
for i in combined.index:
    if '(' in combined.loc[i,'Block_Location']:
        count = count + 1
    else:
        combined = combined.drop(i)
        
print(count)
print(combined.shape)

(455, 11)
439
(439, 11)


In [95]:
# get lat/long coordinates
# you can skip the Point section if you're not mapping!!
for i in combined.index:
    combined.loc[i,'LatLon'] = combined.loc[i,'Block_Location'][combined.loc[i,'Block_Location'].find("(")+1:combined.loc[i,'Block_Location'].rfind(")")]
    combined.loc[i,'Lat'] = float(combined.loc[i,'LatLon'][0:combined.loc[i,'LatLon'].find(',')])
    combined.loc[i,'Lon'] = float(combined.loc[i,'LatLon'][combined.loc[i,'LatLon'].find(',')+2:])
    
geometry = [Point(xy) for xy in zip(combined['Lon'], combined['Lat'])]
combined['geometry'] = geometry
combined_gpd = gpd.GeoDataFrame(combined)

In [96]:
def generateBaseMap(default_location=[37.8721, -122.2578], default_zoom_start=12):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
    return base_map

In [97]:
base_map = generateBaseMap()
base_map

In [98]:
combined_gpd['EVENTDT'] = pd.to_datetime(combined_gpd['EVENTDT'], format='%m/%d/%y %H:%M')
combined_gpd['EVENTTM'] = pd.to_datetime(combined_gpd['EVENTTM'], format='%H:%M')

In [99]:
combined_gpd['month'] = combined_gpd['EVENTDT'].apply(lambda x: x.month)
combined_gpd['week'] = combined_gpd['EVENTDT'].apply(lambda x: x.week)
combined_gpd['day'] = combined_gpd['EVENTDT'].apply(lambda x: x.day)
combined_gpd['hour'] = combined_gpd['EVENTTM'].apply(lambda x: x.hour)

In [100]:
combined_gpd['count'] = [1] * combined_gpd.shape[0]

In [101]:
from folium.plugins import HeatMap
combined_gpd['count'] = 1
base_map = generateBaseMap()
HeatMap(data=combined_gpd[['Lat', 'Lon', 'count']].groupby(['Lat', 'Lon']).sum().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(base_map)

In [102]:
base_map

In [104]:
base_map.add_child(folium.ClickForMarker(popup='Potential Location'))
base_map



# Heatmaps with time... 
Animation doesn't work....

In [83]:
map_hooray = folium.Map(location=[37.8721, -122.2578], zoom_start = 13) 
# Ensure you're handing it floats
combined_gpd['Lat'] = combined_gpd['Lat'].astype(float)
combined_gpd['Lon'] = combined_gpd['Lon'].astype(float)
heat_df = combined_gpd[['Lat', 'Lon']]
# Create weight column, using date
heat_df['Weight'] = combined_gpd['hour']
heat_df['Weight'] = heat_df['Weight'].astype(float)
heat_df = heat_df.dropna(axis=0, subset=['Lat','Lon', 'Weight'])
from folium import plugins

# List comprehension to make out list of lists
heat_data = [[[row['Lat'],row['Lon']] for index, row in heat_df[heat_df['Weight'] == i].iterrows()] for i in range(0,13)]

# Plot it on the map
hm = plugins.HeatMapWithTime(heat_data,auto_play=True,max_opacity=0.8)
hm.add_to(map_hooray)
# Display the map
map_hooray